### Puzzle

https://adventofcode.com/2020/day/20

### Imports

In [20]:
import re

### Load Input

In [1]:
# Store the location of the input directory
data_dir = '../../../data/2020'

# Open the input and store a list of each item as an int
with open(f"{data_dir}/day20_input.txt") as f:
    inputs = f.read().splitlines()

### Part 1

In [2]:
# Initialize a dictionary to store tile numbers and images
tiles_dict = dict()
edges_dict = dict()

In [3]:
def process_tile(tile, tiles_dict, edges_dict):
    # Split the tile into the tile number and the edges
    tile_number = int(tile[0].split(' ')[1].split(':')[0])
    tile_image = tile[1:]
    
    # For each tile, we will store the edges and the entire image
    tiles_dict[tile_number] = dict()
    
    # Get the edges starting from the top moving clockwise
    north_edge = tile_image[0]
    east_edge = ''.join([pixel[-1] for pixel in tile_image])
    south_edge = tile_image[-1][::-1]
    west_edge = ''.join([pixel[0] for pixel in tile_image][::-1])
    
    # Store the edges as a list with the north edges first going counter-clockwise around the image
    edge_list = [south_edge, west_edge, north_edge, east_edge]
    
    # Store the edges in their own dictionary and which tiles have those edges
    for edge in edge_list:
        edge_flipped = edge[::-1]
        if edge in edges_dict.keys():
            edges_dict[edge].append(tile_number)
        
        elif edge_flipped in edges_dict.keys():
            # If the edge has been seen already but flipped, flip the entire image and store          
            edges_dict[edge_flipped].append(tile_number)
        
        else:
            edges_dict[edge] = [tile_number]
            
    # Store both the edge list and the entire image
    tiles_dict[tile_number]['edges'] = edge_list
    tiles_dict[tile_number]['image'] = tile_image
            
    return tiles_dict, edges_dict

In [4]:
tiles_remaining = inputs

# Add all tiles into the dictionary
while tiles_remaining.count('') > 0:
    # Get the next tile from the tiles remaining
    tile = tiles_remaining[:tiles_remaining.index('')]
    
    # Add that tile and its edges to the appropriate dictionaries
    tiles_dict, edges_dict = process_tile(tile, tiles_dict, edges_dict)

    # Remove the most recent tile from the tiles remaining
    tiles_remaining = tiles_remaining[tiles_remaining.index('')+1:]
    
# Add the last tile into the dictionary 
tile = tiles_remaining
tiles_dict, edges_dict = process_tile(tile, tiles_dict, edges_dict)

In [5]:
# Initialize an empty list to store the corner tiles
corners = []

# Check every edge on every tile
for tile_number in tiles_dict.keys():
    # Initialize the number of edges that don't share a border with another image
    naked_edges = 0
    
    # Get just the edges from the tile dictionary
    edge_list = tiles_dict[tile_number]['edges']
    
    # Check the number of naked edges for all edges and flipped edges
    for edge in edge_list:
        if edge not in edges_dict.keys():
            edge = edge[::-1]
        if len(edges_dict[edge]) == 1:
            naked_edges += 1
    
    # Corner pictures will have 2 naked edges
    if naked_edges == 2:
        corners.append(tile_number)

In [6]:
# Multiply the corners together to get the final product
product = 1
for corner in corners:
    product *= corner

In [7]:
product

174206308298779

### Part 2 Helper Functions

In [8]:
def flip_image(image):
    # Flip an image
    flipped_image = [row[::-1] for row in image]
    return flipped_image

In [9]:
def rotate_image_clockwise90(image):
    # Rotate an image 90 degrees counterclockwise
    new_image = []
    for i in range(len(image[0])):
        next_row = ''.join([row[i] for row in image])[::-1]
        new_image.append(next_row)
    return new_image        

In [10]:
def rotate_image(image, n_clockwise_rotations):
    # Rotate an image to the desired orientation
    for i in range(n_clockwise_rotations):
        image = rotate_image_clockwise90(image)
    return image

In [11]:
def remove_border(tile):
    # Remove the border around an image
    tile_borderless = []
    for row_index in range(1, len(tile)-1):
        tile_borderless.append(tile[row_index][1:len(tile[row_index])-1])
    return tile_borderless

### Part 2

In [12]:
# Initialize a list of lists to store a grid of the big picture
big_image = []
# Initialize a list to store each row
big_image_row = []
# Store which tiles are already in the big image
used_tiles = []

# Arbitrarily choose one of the corners to be the top-left corner
top_left_corner = corners[0]

# Rotate it so that the north edge and west edge are naked edges
image = tiles_dict[top_left_corner]['image']

# The north and west edges of the top left corner must be the naked ones
naked_edges = [len(edges_dict[edge]) for edge in tiles_dict[top_left_corner]['edges']]

# Find the index of the second naked edge and rotate the image that many times
rotations_needed = [index for index, edges in enumerate(naked_edges) if edges == 1][1]
image = rotate_image(image, rotations_needed)

# Add the image to the top row and store that tile number as a used tile
big_image_row.append(image)
used_tiles.append(top_left_corner)

# Find the other image with this east edge 
next_east_edge = ''.join([row[-1] for row in image])
next_east_edge_flipped = next_east_edge[::-1]

if next_east_edge in edges_dict.keys():
    next_tiles = [tile for tile in edges_dict[next_east_edge] if tile not in used_tiles]
else:
    next_tiles = [tile for tile in edges_dict[next_east_edge_flipped] if tile not in used_tiles]

In [13]:
# If there are no more unused tiles with that edge, we have reached the end of the row
while len(next_tiles) > 0:
    
    # If there are still unused tiles with that edge, the next tile must fit in the next spot
    next_tile = next_tiles[0]

    # Rotate the tile so that the east edge of the previous tile faces east on this tile and then flip it
    if next_east_edge in tiles_dict[next_tile]['edges']:
        rotations_needed = 4 - tiles_dict[next_tile]['edges'].index(next_east_edge) - 1
        image = flip_image(rotate_image(tiles_dict[next_tile]['image'], rotations_needed))

    else:
        flipped_image = flip_image(tiles_dict[next_tile]['image'])
        if next_east_edge == ''.join([pixel[-1] for pixel in flipped_image]):
            rotations_needed = 0
        elif next_east_edge == flipped_image[0]:
            rotations_needed = 1
        elif next_east_edge == ''.join([pixel[0] for pixel in flipped_image][::-1]):
            rotations_needed = 2
        else:
            rotations_needed = 3
        image = flip_image(rotate_image(flipped_image, rotations_needed))
        
    # Add the correctly-oriented image to the row and add the tile to the list of used tiles
    big_image_row.append(image)
    used_tiles.append(next_tile)
    
    # Find the other image with this east edge 
    next_east_edge = ''.join([row[-1] for row in image])
    next_east_edge_flipped = next_east_edge[::-1]
    
    # Get the next east edge
    if next_east_edge in edges_dict.keys():
        next_tiles = [tile for tile in edges_dict[next_east_edge] if tile not in used_tiles]
    else:
        next_tiles = [tile for tile in edges_dict[next_east_edge_flipped] if tile not in used_tiles]

big_image.append(big_image_row)

In [14]:
# Now get all of the south edges from the row above and find the tile with the corresponding north edge
while len(used_tiles) < len(tiles_dict.keys()):
    
    # Initialize a new row
    big_image_row = []
    
    # Grab the row above
    row_above = big_image[-1]
    
    # Get the south edge of each tile
    for tile in row_above:
        south_edge = tile[-1][::-1]
        south_edge_flipped = south_edge[::-1]
        
        # Find the tile with the corresponding edge
        if south_edge in edges_dict.keys():
            next_tile = [tile for tile in edges_dict[south_edge] if tile not in used_tiles][0]
        else:
            next_tile = [tile for tile in edges_dict[south_edge_flipped] if tile not in used_tiles][0]
           
        # Rotate the tile so that the corresponding edge faces north to match the tile above it
        if south_edge in tiles_dict[next_tile]['edges']:
            rotations_needed = 4 - tiles_dict[next_tile]['edges'].index(south_edge)
            image_upside_down = flip_image(rotate_image(tiles_dict[next_tile]['image'], rotations_needed))
            image = rotate_image(image_upside_down, 2)

        else:
            flipped_image = flip_image(tiles_dict[next_tile]['image'])
            if south_edge == ''.join([pixel[-1] for pixel in flipped_image]):
                rotations_needed = 1
            elif south_edge == flipped_image[0]:
                rotations_needed = 2
            elif south_edge == ''.join([pixel[0] for pixel in flipped_image][::-1]):
                rotations_needed = 3
            else:
                rotations_needed = 0
            image_upside_down = flip_image(rotate_image(flipped_image, rotations_needed))
            image = rotate_image(image_upside_down, 2)

        # Add the correctly-oriented image to the row and add the tile to the list of used tiles
        big_image_row.append(image)
        used_tiles.append(next_tile)
    
    # Add the row of tiles to the big image
    big_image.append(big_image_row)

In [15]:
# Remove border
big_image_borderless = [[remove_border(tile) for tile in big_image_row] for big_image_row in big_image]

In [16]:
# Push all of the images together to form a single list of strings
big_image_joined = []
for row in big_image_borderless:
    big_image_joined.append([''.join([tile[i] for tile in row]) for i in range(len(row[0]))])

big_image = [row for image_row in big_image_joined for row in image_row]

In [54]:
# Create a regex to find all sea monsters
sea_monster = '(#){1}(.){77}#{1}(.){4}(#){2}(.){4}(#){2}(.){4}(#){3}(.){77}(#){1}(.){2}(#){1}(.){2}(#){1}(.){2}(#){1}(.){2}(#){1}(.){2}(#){1}'

# Try flipping and rotating the image
for i in range(2):
    for j in range(4):
        big_image_copy = big_image.copy()
        
        if i == 1:
            big_image_flipped = flip_image(big_image_copy)
        else:
            big_image_flipped = big_image_copy
            
        big_image_flipped_rotated = rotate_image(big_image_flipped, j)
        
        # Turn the list of strings into a single string
        sea = ''.join(big_image_flipped_rotated)

        # Check the string for all sea monsters
        matches = re.finditer(f"(?=({sea_monster}))", sea)
        results = [match.group(1) for match in matches]
        print(f"{i} flips, {j} rotations: {len(results)} sea monsters")

0 flips, 0 rotations: 29 sea monsters
0 flips, 1 rotations: 0 sea monsters
0 flips, 2 rotations: 0 sea monsters
0 flips, 3 rotations: 0 sea monsters
1 flips, 0 rotations: 0 sea monsters
1 flips, 1 rotations: 0 sea monsters
1 flips, 2 rotations: 0 sea monsters
1 flips, 3 rotations: 0 sea monsters


In [55]:
# Manually input the correct number of sea monsters from the output above and count the remaining chop
total_chop = sea.count('#')
non_sea_monster_chop = total_chop - 29*15

In [56]:
non_sea_monster_chop

2409